## Importando bibliotecas necessárias

In [3]:
import os
from pathlib import Path
import pandas as pd
import uuid
from chromadb import PersistentClient
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import chromadb


In [4]:
load_dotenv()

embedding_function = OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_API_BASE"),
    api_type="azure",
    api_version=os.getenv("OPENAI_API_VERSION"),
    model_name=os.environ["EMBEDDING_DEPLOYMENT_NAME"]
)

In [5]:
path_project = Path.cwd().resolve().parents[0]
data_dir = path_project / 'doc' / 'dataframe_apostila.csv'
apostila_dataframe = pd.read_csv(data_dir)

display(apostila_dataframe.head())

,Capitulo,Seção,Contexto
0,CAPÍTULO 1 COMO APRENDER PYTHON,1.2 SOBRE OS EXERCÍCIOS,1 COMO APRENDER PYTHON \n1
1,CAPÍTULO 1 COMO APRENDER PYTHON,1.3 TIRANDO DÚVIDAS E INDO ALÉM,"Para tirar dúvidas de exercícios, ou de Python..."
2,CAPÍTULO 2 O QUE É PYTHON,2.2 BREVE HISTÓRIA,Python foi criada em 1990 por Guido Van Rossum...
3,CAPÍTULO 2 O QUE É PYTHON,2.3 INTERPRETADOR,Você provavelmente já ouviu ou leu em algum lu...
4,CAPÍTULO 2 O QUE É PYTHON,2.4 QUAL VERSÃO UTILIZAR?,"Para quem está começando, a primeira dúvida na..."


In [6]:
def prepare_data(df: pd.DataFrame) -> tuple[list, list, list]:
    """
    Prepare the lists of ids, documents and metadatas for all
    items in a given dataframe.
    
    Parameters
    ----------
    df : pd.DataFrame
        Review data for the books
        
    Returns
    -------
    tuple[list, list, list]
        
    """

    documents = []
    metadatas = []
    ids = []

    for ind, book in enumerate(df['Seção']):
        temp = df.iloc[ind]
        metadata = {
            'Session': temp['Seção'],
            'Chapter': temp['Capitulo'],
        }

        documents.append(str(temp['Contexto']))
        metadatas.append(metadata)
        ids.append(uuid.uuid4().hex)
        
    return ids, documents, metadatas

In [7]:
ids, documents, metadatas = prepare_data(df=apostila_dataframe)

In [8]:
ids[:5]

['9bab639c5d9c4ce997fb25098f8e1eda',
 'e668aeb5ac134aa69dfc6b405da990fd',
 '5ebb7bd385d941bdb1bc182822bf2343',
 '749b69c766204978b35656795e05af01',
 '09a22ac9aa1948e9aa2095b88798c811']

In [9]:
metadatas[:10]

[{'Session': '1.2 SOBRE OS EXERCÍCIOS',
  'Chapter': 'CAPÍTULO 1 COMO APRENDER PYTHON'},
 {'Session': '1.3 TIRANDO DÚVIDAS E INDO ALÉM',
  'Chapter': 'CAPÍTULO 1 COMO APRENDER PYTHON'},
 {'Session': '2.2 BREVE HISTÓRIA', 'Chapter': 'CAPÍTULO 2 O QUE É PYTHON'},
 {'Session': '2.3 INTERPRETADOR', 'Chapter': 'CAPÍTULO 2 O QUE É PYTHON'},
 {'Session': '2.4 QUAL VERSÃO UTILIZAR?',
  'Chapter': 'CAPÍTULO 2 O QUE É PYTHON'},
 {'Session': '2.5 DOWNLOAD', 'Chapter': 'CAPÍTULO 2 O QUE É PYTHON'},
 {'Session': '2.6 CPYTHON, JYTHON, IRONPYTHON?',
  'Chapter': 'CAPÍTULO 2 O QUE É PYTHON'},
 {'Session': '2.5 DOWNLOAD', 'Chapter': 'CAPÍTULO 2 O QUE É PYTHON'},
 {'Session': '2.7 PEP - O QUE SÃO E PRA QUE SERVEM',
  'Chapter': 'CAPÍTULO 2 O QUE É PYTHON'},
 {'Session': '2.8 ONDE USAR E OBJETIVOS',
  'Chapter': 'CAPÍTULO 2 O QUE É PYTHON'}]

In [10]:
documents[:5]

['1 COMO APRENDER PYTHON \n1',
 'Para tirar dúvidas de exercícios, ou de Python em geral, recomendamos o fórum do GUJ Respostas:\nhttp://www.guj.com.br\nLá sua dúvida será respondida prontamente. O GUJ foi fundado por desenvolvedores da Caelum e\nhoje conta com mais de um milhão de mensagens.\nO principal recurso oficial para encontrar documentação, tutoriais e até mesmo livros sobre Python é\na Python Software Foundation (PSF):\nhttps://www.python.org/\nDestacamos também a página da comunidade no Brasil:\nhttps://python.org.br/\nHá também fóruns oficiais da comunidade:\nhttps://python-forum.io/ (inglês)\nhttps://python.org.br/lista-de-discussoes/ (português)\nFora isso, sinta-se à vontade para entrar em contato com seu instrutor para tirar todas as dúvidas que\nsurgirem durante o curso.\nSe o que você está buscando são livros de apoio, sugerimos conhecer a editora Casa do Código:\nhttps://www.casadocodigo.com.br/\nHá também cursos online que vão ajudá-lo a ir além, com muita interação

In [11]:
def get_collection(collection_name: str, embedding_function: OpenAIEmbeddingFunction) -> PersistentClient:
    """
    Search for the collection based on its name. Creates if
    it does not already exists.
    
    Parameters
    ----------
    collection_name : str
        Name of the collection to be retrieved.
    embedding_function : OpenAIEmbeddingFunction
        Type of embedding to be used in the collection.
        
    Returns
    -------
    PersistentClient
        Collection in persistent path
    """
    
    collection_path = os.getcwd().replace("notebooks", "db")
    client = chromadb.PersistentClient(path=collection_path)

    try:
        collection = client.get_collection(name=collection_name, embedding_function=embedding_function)
        print(f"Coleção {collection_name} recuperada com sucesso.")
        print(f"Numero de documentos na coleção: {collection.count()}")

    except:
        collection = client.create_collection(name=collection_name, embedding_function=embedding_function)
        print(f"Coleção {collection_name} criada com sucesso.")
        
    return collection

In [12]:
collection = get_collection(collection_name="apostila_python", embedding_function=embedding_function)

Coleção apostila_python criada com sucesso.


In [13]:
def add_docs_to_collection(collection: PersistentClient, documents: list, metadatas: list, ids: list):
    """
    Search for the collection based on its name.
    
    Parameters
    ----------
    collection : PersistentClient
        Collection to add the files.
    documents : list
        List with the contents of the documents.
    metadatas : list
        List with the metadatas of the documents.
    ids : list
        List with unique ids of the documents.    
        
    Returns
    -------
    PersistentClient
        Collection in persistent path
    """
    
    print(f"Adding documents to collection {collection.name}")
    collection.add(documents=documents, metadatas=metadatas, ids=ids)
    print("Process ended.")

In [14]:
add_docs_to_collection(collection=collection, documents=documents, metadatas=metadatas, ids=ids)

Adding documents to collection apostila_python
Process ended.


In [15]:
print(f"Número de documentos na coleção: {collection.count()}")

Número de documentos na coleção: 120


In [16]:
def search_embeddings(collection: PersistentClient, query: str, filters: dict = {}, k: int = 10) -> tuple[list, list]:
    """
    Retrive context for a given query.
    
    Parameters
    ----------
    collection : PersistentClient
        Collection to retrive the documents.
    query : str
        Message from the user
    filters : dict
        Filters for the vector store, by default an
        empty dictionary
    k : int
        Number of documents to retrieve, by default 20
        
    Returns
    -------
    tuple[list, list]
        Retrived docs contents and metadatas
    """
    
    results = collection.query(
        query_texts=[query],
        n_results=k,
        where=filters,
    )
    
    contents = results["documents"][0]
    metadatas = results["metadatas"][0]
    
    return contents, metadatas

In [17]:
query = "Como aprender Python?"

contents, metadatas = search_embeddings(collection=collection, query=query)
print(f"Numero de documentos recebidos {len(contents)}\n---------------------------------\n\n")
print(f"Primeiros cinco documentos:\n\n")

for i in range(5):
    print(f"Capítulo: {metadatas[i]['Chapter']}")
    print(f"Seção: {metadatas[i]['Session']}")
    print(f"Seção: {contents[i]}")


Numero de documentos recebidos 10
---------------------------------


Primeiros cinco documentos:


Capítulo: CAPÍTULO 1 COMO APRENDER PYTHON
Seção: 1.2 SOBRE OS EXERCÍCIOS
Seção: 1 COMO APRENDER PYTHON 
1
Capítulo: CAPÍTULO 1 COMO APRENDER PYTHON
Seção: 1.3 TIRANDO DÚVIDAS E INDO ALÉM
Seção: Para tirar dúvidas de exercícios, ou de Python em geral, recomendamos o fórum do GUJ Respostas:
http://www.guj.com.br
Lá sua dúvida será respondida prontamente. O GUJ foi fundado por desenvolvedores da Caelum e
hoje conta com mais de um milhão de mensagens.
O principal recurso oficial para encontrar documentação, tutoriais e até mesmo livros sobre Python é
a Python Software Foundation (PSF):
https://www.python.org/
Destacamos também a página da comunidade no Brasil:
https://python.org.br/
Há também fóruns oficiais da comunidade:
https://python-forum.io/ (inglês)
https://python.org.br/lista-de-discussoes/ (português)
Fora isso, sinta-se à vontade para entrar em contato com seu instrutor para tirar 